In [1]:
pip install requests beautifulsoup4 pandas

In [1]:
!pip install requests beautifulsoup4 pandas openpyxl

In [4]:
import pandas as pd


df = pd.read_excel("Car_annoucement.xlsx")

df_limited = df.head(100)

df_limited.to_excel("your_file_limited.xlsx", index=False)

df_limited


,Prices,Countries,Sellers,Brand,Model,Quantity,day,month,year
0,28000000,Côte d'Ivoire,Issa Auto,Toyota,Highlander 2022,1,18,Avril,2025
1,34000000,Côte d'Ivoire,Issa Auto,Toyota,Rav4 2023,1,18,Avril,2025
2,16700000,Côte d'Ivoire,Js Voiture,Hyundai,Sonata 2020,1,18,Avril,2025
3,160000000,Côte d'Ivoire,Rephidim Auto Ci,Rover,Range Rover Vogue 2024,1,18,Avril,2025
4,52700000,Côte d'Ivoire,Js Voiture,Infiniti,QX80 2021,1,18,Avril,2025
...,...,...,...,...,...,...,...,...,...
95,2900000,Côte d'Ivoire,Auto Succes,Volkswagen,Golf 2005,1,18,Avril,2025
96,6800000,Côte d'Ivoire,Na Services,Hyundai,Elantra 2016,1,18,Avril,2025
97,32000000,Côte d'Ivoire,Na Services,Toyota,Tacoma 2023,1,18,Avril,2025
98,19800000,Côte d'Ivoire,First Parc Auto,Honda,CR-V 2021,1,18,Avril,2025


In [9]:
print("Columns in dataframe:")
print(df_limited.columns.tolist())

Columns in your DataFrame:
['Prices', 'Countries', 'Sellers', 'Brand', 'Model', 'Quantity', 'day', 'month', 'year']


In [7]:
import pyodbc
import pandas as pd
from unidecode import unidecode  # For handling special characters

# 1. Install unidecode 
try:
    from unidecode import unidecode
except:
    !pip install unidecode
    from unidecode import unidecode

# 2. Database Connection
conn = pyodbc.connect(
    'DRIVER={SQL Server};'
    'SERVER=PC-Fatim;'
    'DATABASE=chette;'
    'Trusted_Connection=yes;'
)
cursor = conn.cursor()



# 4. Data Preparation with Special Character Handling
def prepare_data(df):
    df = df.copy()
    
    # Convert special characters to ASCII (Côte d'Ivoire -> Cote d'Ivoire)
    text_cols = ['Brand', 'Model', 'Countries', 'Sellers']
    for col in text_cols:
        df[col] = df[col].apply(lambda x: unidecode(str(x)))
    
    # Standard cleaning
    for col in text_cols + ['month']:
        df[col] = df[col].str.strip().str.title()
    
    # Numeric conversions
    df['Prices'] = pd.to_numeric(df['Prices'], errors='coerce')
    df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce').fillna(1).astype(int)
    df['day'] = pd.to_numeric(df['day'], errors='coerce').fillna(1).astype(int)
    df['year'] = pd.to_numeric(df['year'], errors='coerce').fillna(2023).astype(int)
    
    # Month conversion
    month_map = {
        'Janvier': 1, 'Fevrier': 2, 'Mars': 3, 'Avril': 4,
        'Mai': 5, 'Juin': 6, 'Juillet': 7, 'Aout': 8,
        'Septembre': 9, 'Octobre': 10, 'Novembre': 11, 'Decembre': 12
    }
    df['month'] = df['month'].map(month_map).fillna(1).astype(int)
    
    return df.dropna()

df_clean = prepare_data(df_limited)

# 5. Insert Dimensions
def insert_unique(table, column, values):
    """Insert unique values into a table"""
    values = list(set(values))  # Get unique values
    for val in values:
        cursor.execute(f"INSERT INTO {table} ({column}) VALUES (?)", val)
    conn.commit()

# Insert all unique values
insert_unique('brand', 'brand_name', df_clean['Brand'])
insert_unique('country', 'country_name', df_clean['Countries'])
insert_unique('seller', 'seller_name', df_clean['Sellers'])

# Insert models with brand relationships
brand_ids = {row[0]: row[1] for row in cursor.execute("SELECT brand_name, brand_id FROM brand")}
for model, brand in df_clean[['Model', 'Brand']].drop_duplicates().itertuples(index=False):
    cursor.execute("INSERT INTO model (model_name, brand_id) VALUES (?, ?)", model, brand_ids[brand])
conn.commit()

# Insert dates
for year, month, day in df_clean[['year', 'month', 'day']].drop_duplicates().itertuples(index=False):
    cursor.execute("INSERT INTO dim_date (year, month, day) VALUES (?, ?, ?)", year, month, day)
conn.commit()

# 6. Insert Facts
# Get all ID mappings
model_ids = {row[0]: row[1] for row in cursor.execute("SELECT model_name, model_id FROM model")}
country_ids = {row[0]: row[1] for row in cursor.execute("SELECT country_name, country_id FROM country")}
seller_ids = {row[0]: row[1] for row in cursor.execute("SELECT seller_name, seller_id FROM seller")}
date_ids = {(row[0], row[1], row[2]): row[3] for row in cursor.execute("SELECT year, month, day, dim_date_id FROM dim_date")}

# Insert all fact records
for _, row in df_clean.iterrows():
    cursor.execute("""
        INSERT INTO sale_fact (
            model_id, country_id, dim_date_id, 
            seller_id, quantity, sale_cost
        ) VALUES (?, ?, ?, ?, ?, ?)
        """,
        model_ids[row['Model']],
        country_ids[row['Countries']],
        date_ids[(row['year'], row['month'], row['day'])],
        seller_ids[row['Sellers']],
        row['Quantity'],
        float(row['Prices'])
    )

conn.commit()
print("✅ ALL DATA SUCCESSFULLY IMPORTED")
conn.close()

✅ ALL DATA SUCCESSFULLY IMPORTED


In [ ]:
# code to delete existing data in reverse order in case of error
cursor.execute("DELETE FROM sale_fact")
cursor.execute("DELETE FROM dim_date")
cursor.execute("DELETE FROM seller")
cursor.execute("DELETE FROM country")
cursor.execute("DELETE FROM model")
cursor.execute("DELETE FROM brand")
conn.commit()